In [1]:
import numpy as np
import tensorflow as tf
from copy import deepcopy

import import_ipynb
from QBert import train_utils, models

from tqdm.notebook import tqdm


importing Jupyter notebook from C:\Users\LGCNS\Documents\GitHub\Q_Bert\QBert\train_utils.ipynb
importing Jupyter notebook from C:\Users\LGCNS\Documents\GitHub\Q_Bert\QBert\models.ipynb


## 해당 파일은 bert.run_pretraining.run_bert_pretrain을 구현하는 것을 목표로 한다.

 - Parameter는 FLAG 형식에서 직접 정의해주는 방식으로 변경하고, Main에서 직접 정의하도록 한다.

## Strategy

In [2]:
tf.config.list_physical_devices() # device 확인

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

In [3]:
# Strategy 정의

distribution_strategy = 'mirrored' # 'tpu'
num_gpus = 0
all_reduce_alg = None

if distribution_strategy == 'tpu' :
    tpu_address = ""
else :
    tpu_address = None



strategy = train_utils.get_distribution_strategy(
                  distribution_strategy=distribution_strategy,
                  num_gpus=num_gpus,
                  all_reduce_alg=all_reduce_alg,
                  tpu_address=tpu_address)

print ('\n장치의 수: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)

장치의 수: 1


## Input_Files

In [4]:
filenames = ['./Test_Examples.tfrecords', './Test_Examples.tfrecords']

# Create a description of the features.
feature_description = {
    'input_ids': tf.io.FixedLenFeature([256], tf.int64),
    'segment_ids': tf.io.FixedLenFeature([256], tf.int64),
    'input_mask': tf.io.FixedLenFeature([256], tf.int64),
    'masked_lm_positions': tf.io.FixedLenFeature([255], tf.int64),
    'masked_lm_ids': tf.io.FixedLenFeature([255], tf.int64),
    'masked_lm_weights': tf.io.FixedLenFeature([255], tf.float32),
    'next_sentence_labels': tf.io.FixedLenFeature([1], tf.int64),
}

# keys = feature_description.keys()

In [5]:
def _parse_function(example_proto):
  # Parse the input `tf.train.Example` proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, feature_description)

BUFFER_SIZE = 672

BATCH_SIZE_PER_REPLICA = 64
GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

EPOCHS = 10

In [6]:

with strategy.scope():

    train_dataset = tf.data.TFRecordDataset(filenames)
    train_dataset = train_dataset.map(_parse_function)
    train_dataset_iters = train_dataset.shuffle(BUFFER_SIZE).batch(1) 
    
    train_dist_dataset = strategy.experimental_distribute_dataset(train_dataset_iters)


## Bert_Config

In [7]:
# sub_model (core_model 필요 Config)

vocab_size = 32000 # 
hidden_size = 768 # Transformer hidden Layers
type_vocab_size = 12 #: The number of types that the 'type_ids' input can take.
num_layers = 12
num_attention_heads = 12
max_seq_length = 512
dropout_rate = .1
# attention_dropout_rate = .1
inner_dim = 3072
# hidden_act = 'gelu'
initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02)

# Pretrain Model 필요 Config
max_predictions_per_seq = 255


## Model Define

In [8]:
with strategy.scope() :

    model, sub_model = models.get_bert_models_fn(vocab_size
                 , hidden_size
                 , type_vocab_size
                 , num_layers
                 , num_attention_heads
                 , max_seq_length
                 , max_predictions_per_seq
                 , dropout_rate
                 , inner_dim 
                 , initializer)

In [9]:
for inputs in train_dataset_iters.take(1) :
    print(model([inputs['input_ids'], inputs['input_mask'], inputs['segment_ids']]))

## Traning Config

In [10]:
vocab_size = 32000
max_seq_len = 130 # 512
num_layers = 6 # 12
dff = 384 * 2 # 768 * 4
d_model = 384 # 768
num_heads = 6 # 12
dropout = .1
name = 'qbert_210603'